In [2]:
from JPM_Task_1_Solution import Task_1_Solution
import pandas as pd
from datetime import datetime
from typing import List


def Task_2_Solution(set_quantity: float, injection_dates: List[datetime], withdrawal_dates: List[datetime], inj_with_rate: float, max_storage: float, storage_rate: float) -> float:
    """
    Solution to TASK 2 of the JPM Quantitative Research Course.

    Args:
        set_quantity: The quantity(in millions of MMBtu) of natural gas injected/withdrawn in every transaction
        injection_dates: A list of datetime objects corresponding to dates when the gas is injected
        withdrawal_dates: A list of datetime objects corresponding to dates when the gas is withdrawn
        inj_with_rate: The rate of injecting/withdrawing natural gas per million MMBtu
        max_storage: The maximum volume(in millions of MMBtu) of gas that can be stored
        storage_rate: The rate of storing natural gas per month

    Returns:
        The pricing/valuation of the contract(in dollars)

    """

    buy_prices = pd.Series((Task_1_Solution(date) for date in injection_dates), index=injection_dates).sort_index()
    sell_prices = pd.Series((Task_1_Solution(date) for date in withdrawal_dates), index=withdrawal_dates).sort_index()
    revenue = sum(sell_prices * set_quantity * 1000000)
    op_expense = sum(buy_prices * set_quantity * 1000000)
    
    current_capacity = 0.0
    storage_cost = 0.0
    all_orders_placed = pd.concat([buy_prices,sell_prices]).sort_index()
    prev_year = all_orders_placed.index[0].year
    prev_month = all_orders_placed.index[0].month

    for order_date in all_orders_placed.index:
        if current_capacity > 0:
            storage_cost = storage_cost + storage_rate * ((order_date.year * 12 + order_date.month) - (prev_year * 12 + prev_month))
        
        if order_date in injection_dates:
            current_capacity = current_capacity + set_quantity
            injection_dates.remove(order_date)
        else:
            current_capacity = current_capacity - set_quantity
            withdrawal_dates.remove(order_date)

        if current_capacity < 0:
            raise ValueError("Cannot withdraw gas when there is no gas injected in storage!")

        if current_capacity > max_storage:
            raise ValueError("Current capacity exceeds maximum storage!")
        
        prev_month = order_date.month
        prev_year = order_date.year
        
    valuation = revenue - op_expense - storage_cost - (all_orders_placed.size * set_quantity * inj_with_rate)
    return valuation
        

Task_2_Solution(1.0, [datetime(2020,11,23),datetime(2020,12,23)], [datetime(2021,4,28),datetime(2021,3,28)], 10000.0, 5.0, 100000.0)

/Users/aryasalian/MyProjects-github/JPM Quantitative Research Course Projects/Task 2/JPM_Task_1_Solution.py:5: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data = pd.read_csv('/Users/aryasalian/JPM Quant Course/Task 2/Nat_Gas OG copy.csv', names = ['Dates','Prices'], header = 0, parse_dates=['Dates'], index_col= 'Dates')


860000.3843742907